In [1]:
from __future__ import print_function

import os
from ftplib import FTP
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
# import nltk
from nltk.corpus import stopwords
from nltk import tokenize
import string
import numpy as np
import re
import glob
from io import open
from collections import Counter


#do this stuff? ionno
# import nltk 
# nltk.download("stopwords")
# nltk.download('punkt')

### Make txt files (of raw string)

In [2]:
local_dir = '/Users/rhaggerty/remote/bills/tx/data/'

def is_empty(file):
    file_size=os.stat(file).st_size
    if file_size == 0:
        print("The file {} is empty".format(file))
        return True
    return False


#make sure html file isn't empty, if it is, remove it
def prettify_soup(doc_name):
#     'HB00098I'
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    output_dir = os.path.join(local_dir, 'text/')
    
    if (is_empty(doc_name)):
        os.remove(doc_name)
    else:
        with open(doc_name, 'rt') as in_file:
            soup = BeautifulSoup(in_file, 'html.parser') 
            pretty_text = soup.get_text(' ', strip=True)
#             twopretty = pretty_text.prettify()
            with open(output_dir + bill_no + '.txt', 'w+', encoding='utf-8') as out_file:
#                 out_file.write(pretty_text.encode('utf8'))
                out_file.write(pretty_text)

In [3]:
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    prettify_soup(doc_name)

### Get summaries

In [81]:
summary_list = []
bill_no_list = []
summary_dict = {}
for doc_name in glob.glob(local_dir + 'text/*.txt'):
    bill_no = doc_name.split('/')[-1].rstrip('.txt')
    with open(doc_name, 'rt') as in_file:
        full_text = in_file.read()
        sentences_list = tokenize.sent_tokenize(full_text)
        summary_tag = re.compile('A BILL TO BE ENTITLED')
        summary_sentences = filter(summary_tag.search, sentences_list)
        for sentence in summary_sentences:
            cleaned_sentence = re.sub('.*A BILL TO BE ENTITLED AN ACT relating to ', '', sentence)
    summary_list.append(cleaned_sentence)
    bill_no_list.append(bill_no)
bill_summaries = zip(bill_no_list, summary_list)
for bill, summary in bill_summaries:
    summary_dict[bill] = summary
print(summary_dict)

{'HB00197I': u'the bilingual education allotment provided under the public school finance system.', 'HB00284I': u'the designation of a common area for on-premise consumption by a holder of certain alcoholic beverage permits and licenses.', 'HB00233I': u"medical causation narrative reports created under the Texas Workers' Compensation Act.", 'HB00105I': u'the minimum wage.', 'HB00150I': u'the model training curriculum and required training for certain school district peace officers and school resource officers.', 'HB00058I': u'the establishment of a tax credit scholarship and educational expense assistance program.', 'HB00111H': u'general obligation bonds issued by political subdivisions.', 'HB00005I': u'a limitation on increases in the appraised value for ad valorem tax purposes of commercial or industrial real property.', 'HB00042I': u'the selection of the board of directors of an appraisal district; authorizing the imposition of a fee.', 'HB00372I': u'an audit of funds received by ce

In [15]:
days_file = open('/Users/rhaggerty/remote/bills/tx/data/text/HB00001H.txt','rt')
thing = days_file.read()
print(thing)

85(1) HB 1 - House Committee Report version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10968 MCK-D By: Gonzales of Williamson, Burkett, Raymond, H.B. No. 1 Flynn, Thompson of Harris A BILL TO BE ENTITLED AN ACT relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION 1.  Section 151.004, Occupations Code, is amended to read as follows: Sec. 151.004.  APPLICATION OF SUNSET ACT.  The Texas Medical Board is subject to Chapter 325, Government Code (Texas Sunset Act).  Unless continued in existence as provided by that chapter, the board is abolished and this subtitle and Chapters 204, 205, 206, 601, 602, 603, and 604 expire September 1, 2019 [ 2017 ]. SECTION 2.  Section 501.005, Occupations Code, is amended to read as follows: Sec. 501.005.  APPLICATION OF SUNSET ACT.  The Texas State Board of Examiners of Psy

In [68]:
heyu_guyz_list = []
wc_list = []
for heyu in summary_list:
    for guyz in heyu:
        this = re.sub('.*A BILL TO BE ENTITLED AN ACT relating to ', '', guyz)
        heyu_guyz_list.append(this)
        words = this.split()
        non_stop = [w for w in words if not w in stopwords.words("english")]
        wc_list.append(non_stop)
wc_list

[[u'avoiding',
  u'abolishment',
  u'certain',
  u'agencies',
  u'subject',
  u'Texas',
  u'Sunset',
  u'Act',
  u'September',
  u'1,',
  u'2017.'],
 [u'avoiding',
  u'abolishment',
  u'certain',
  u'agencies',
  u'subject',
  u'Texas',
  u'Sunset',
  u'Act',
  u'September',
  u'1,',
  u'2017.'],
 [u'avoiding',
  u'abolishment',
  u'certain',
  u'agencies',
  u'subject',
  u'Texas',
  u'Sunset',
  u'Act',
  u'September',
  u'1,',
  u'2017.'],
 [u'repeal',
  u'certain',
  u'riders',
  u'Texas',
  u'Medical',
  u'Board',
  u'Texas',
  u'State',
  u'Board',
  u'Examiners',
  u'Psychologists',
  u'General',
  u'Appropriations',
  u'Act',
  u'contingent',
  u'upon',
  u'approval',
  u'certain',
  u'Acts',
  u'continuing',
  u'agencies',
  u'85th',
  u'Regular',
  u'Session.'],
 [u'repeal',
  u'certain',
  u'riders',
  u'Texas',
  u'Medical',
  u'Board',
  u'Texas',
  u'State',
  u'Board',
  u'Examiners',
  u'Psychologists',
  u'General',
  u'Appropriations',
  u'Act',
  u'contingent',
  u'u

In [71]:
word_set = set([i[0] for i in wc_list])
word_set

{u'Legislative',
 u'Medicaid',
 u'abatement',
 u'abolition',
 u'acquisition',
 u'actions',
 u'ad',
 u'additional',
 u'adjustment',
 u'administration',
 u'admission',
 u'adoption',
 u'advance',
 u'age',
 u'allocating',
 u'allocation',
 u'annual',
 u'applicability',
 u'application',
 u'appointment',
 u'appropriation',
 u'appropriations',
 u'assessment',
 u'assistance',
 u'assisting',
 u'audit',
 u'authority',
 u'authorizing',
 u'automatic',
 u'availability',
 u'average',
 u'avoiding',
 u'bilingual',
 u'burden',
 u'calculation',
 u'cardiac',
 u'carrying',
 u'certain',
 u'civil',
 u'class',
 u'collection',
 u'collection,',
 u'compensation',
 u'compensatory',
 u'composition',
 u'computation',
 u'conflicts',
 u'constitutional',
 u'construction',
 u'cost-of-living',
 u'coverage',
 u'creation',
 u'cybersecurity',
 u'date',
 u'days',
 u'dedicating',
 u'defense',
 u'deferred',
 u'definition',
 u'deposit',
 u'designation',
 u'determination',
 u'development',
 u'disclosure',
 u'disposal',
 u'disse

In [77]:
word_list_two = [i[0] for i in wc_list]
favs = Counter(word for word in word_list_two)
favs.most_common()

[(u'authority', 19),
 (u'exemption', 17),
 (u'salary', 12),
 (u'public', 10),
 (u'limitation', 10),
 (u'use', 10),
 (u'creation', 10),
 (u'reporting', 10),
 (u'regulation', 9),
 (u'selection', 8),
 (u'appropriation', 8),
 (u'eligibility', 8),
 (u'calculation', 8),
 (u'maternal', 8),
 (u'making', 7),
 (u'administration', 7),
 (u'establishment', 7),
 (u'rate', 6),
 (u'cost-of-living', 5),
 (u'health', 5),
 (u'extension', 5),
 (u'study', 5),
 (u'prohibiting', 5),
 (u'repeal', 5),
 (u'prevention', 5),
 (u'pregnancy-related', 4),
 (u'restrictions', 4),
 (u'applicability', 4),
 (u'reduction', 4),
 (u'general', 4),
 (u'prosecution', 4),
 (u'membership', 4),
 (u'constitutional', 4),
 (u'reappraisal', 3),
 (u'school', 3),
 (u'automatic', 3),
 (u'requirement', 3),
 (u'authorizing', 3),
 (u'prohibited', 3),
 (u'extensions', 3),
 (u'disclosure', 3),
 (u'requiring', 3),
 (u'small-sized', 3),
 (u'lobbying', 3),
 (u'avoiding', 3),
 (u'deferred', 3),
 (u'certain', 3),
 (u'municipal', 3),
 (u'grant', 3